### Comparing Models and Vectorization Strategies for Text Classification

This Try-It activity focuses on weighing the positives and negatives of different estimators and vectorization strategies for a text classification problem.  In order to consider each of these components, you should make use of the `Pipeline` and `GridSearchCV` objects in Scikit-Learn to try different combinations of vectorizers with different estimators.  For each of these, you also want to use the `.cv_results_` to examine the time for the estimator to fit the data.

### The Data

The dataset below is from [kaggle]() and contains a dataset named the "ColBert Dataset" created for this [paper](https://arxiv.org/pdf/2004.12765.pdf).  You are to use the text column to classify whether or not the text was humorous.  It is loaded and displayed below.


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.preprocessing import StandardScaler
import re

import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\prove\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\prove\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\prove\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
df = pd.read_csv('data/dataset-minimal.csv')

In [3]:
df.head()

,text,humor
0,"Joe biden rules out 2020 bid: 'guys, i'm not r...",False
1,Watch: darvish gave hitter whiplash with slow ...,False
2,What do you call a turtle without its shell? d...,True
3,5 reasons the 2016 election feels so personal,False
4,"Pasco police shot mexican migrant from behind,...",False


In [4]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['humor'], test_size=0.2, random_state=42)

In [5]:
# Create a list of stop words
stop_words = set(stopwords.words('english'))

# Create a stemmer and lemmatizer
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

#### Task


**Text preprocessing:** As a pre-processing step, perform both `stemming` and `lemmatizing` to normalize your text before classifying. For each technique use both the `CountVectorize`r and `TfidifVectorizer` and use options for stop words and max features to prepare the text data for your estimator.

**Classification:** Once you have prepared the text data with stemming lemmatizing techniques, consider `LogisticRegression`, `DecisionTreeClassifier`, and `MultinomialNB` as classification algorithms for the data. Compare their performance in terms of accuracy and speed.

Share the results of your best classifier in the form of a table with the best version of each estimator, a dictionary of the best parameters and the best score.

In [6]:
# Define a function to preprocess the text
def preprocess_text(text):
    """
    Preprocesses text using stemming and lemmatization, stop word removal, lowercasing,
    and punctuation removal.

    Args:
        text (str): The text to preprocess.

    Returns:
        str: The preprocessed text.
    """
    text = text.lower()  # Lowercase
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    words = text.split()
    words = [stemmer.stem(lemmatizer.lemmatize(word)) for word in words if word not in stop_words]
    return ' '.join(words)

In [7]:
# Define the pipelines
pipelines = [
    # CountVectorizer + Logistic Regression
    Pipeline([
        ('vectorizer', CountVectorizer(stop_words='english', max_features=1500, max_df=0.75)),
        ('scaler', StandardScaler(with_mean=False)),
        ('classifier', LogisticRegression(random_state=42))
    ]),
    # TfidfVectorizer + Logistic Regression
    Pipeline([
        ('vectorizer', TfidfVectorizer(stop_words='english', max_features=1500, max_df=0.75)),
        ('scaler', StandardScaler(with_mean=False)),
        ('classifier', LogisticRegression(random_state=42))
    ]),
    # CountVectorizer + Decision Tree
    Pipeline([
        ('vectorizer', CountVectorizer(stop_words='english', max_features=1500, max_df=0.75)),
        ('scaler', StandardScaler(with_mean=False)),
        ('classifier', DecisionTreeClassifier(random_state=42))
    ]),
    # TfidfVectorizer + Decision Tree
    Pipeline([
        ('vectorizer', TfidfVectorizer(stop_words='english', max_features=1500, max_df=0.75)),
        ('scaler', StandardScaler(with_mean=False)),
        ('classifier', DecisionTreeClassifier(random_state=42))
    ]),
    # CountVectorizer + Bayes
    Pipeline([
        ('vectorizer', CountVectorizer(stop_words='english', max_features=1500, max_df=0.75)),
        ('scaler', StandardScaler(with_mean=False)),
        ('classifier', MultinomialNB())
    ]),
    # TfidfVectorizer + Bayes
    Pipeline([
        ('vectorizer', TfidfVectorizer(stop_words='english', max_features=1500, max_df=0.75)),
        ('scaler', StandardScaler(with_mean=False)),
        ('classifier', MultinomialNB())
    ]),
]

In [8]:
param_grids = [
    # CountVectorizer + Logistic Regression
    {'classifier__C': [0.3, 0.4, 0.5]},
    # TfidfVectorizer + Logistic Regression
    {'classifier__C': [0.15, 0.2, 0.25]},
    # CountVectorizer + Decision Tree
    {'classifier__max_depth': [200, 220, 250], 'classifier__min_samples_split': [50, 60, 70]},
    # TfidfVectorizer + Decision Tree
    {'classifier__max_depth': [150, 175, 200], 'classifier__min_samples_split': [70, 80, 90]},
    # CountVectorizer + Bayes
    {'classifier__alpha': [0.004, 0.005, 0.006]},
    # TfidfVectorizer + Bayes
    {'classifier__alpha': [0.004, 0.005, 0.006]}
]

### Results will be in the format below:

In [9]:
# Set up a dictionary to store the results
results = {
    'model': [
        'Logistic Regression (CountVectorizer)',
        'Logistic Regression (TfidfVectorizer)',
        'Decision Tree (CountVectorizer)',
        'Decision Tree (TfidfVectorizer)',
        'Naive Bayes (CountVectorizer)',
        'Naive Bayes (TfidfVectorizer)'
    ],
    'best_params': ['' for _ in range(6)],
    'best_score': ['' for _ in range(6)],
    'mean_fit_time': ['' for _ in range(6)]
}

In [10]:
pd.set_option('display.max_colwidth', None)  # Set to None for unlimited width

# Train and evaluate each pipeline with GridSearchCV
for i, (pipeline, param_grid) in enumerate(zip(pipelines, param_grids)):
    grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy')
    grid_search.fit(X_train, y_train)

    results['best_params'][i] = str(grid_search.best_params_)
    results['best_score'][i] = grid_search.best_score_
    results['mean_fit_time'][i] = grid_search.cv_results_['mean_fit_time'][grid_search.best_index_]
    print(f"{results['model'][i]}:")
    print(f"  Best params: {results['best_params'][i]}")
    print(f"  Best cross-validation score: {results['best_score'][i]:.4f}")
    print(f"  Mean fit time: {results['mean_fit_time'][i]:.2f}s")
    print()


# Create a Pandas DataFrame from the results
results_df = pd.DataFrame(results).set_index('model')

# Print the results
results_df

Logistic Regression (CountVectorizer):
  Best params: {'classifier__C': 0.3}
  Best cross-validation score: 0.8410
  Mean fit time: 0.54s

Logistic Regression (TfidfVectorizer):
  Best params: {'classifier__C': 0.15}
  Best cross-validation score: 0.8439
  Mean fit time: 0.53s

Decision Tree (CountVectorizer):
  Best params: {'classifier__max_depth': 250, 'classifier__min_samples_split': 70}
  Best cross-validation score: 0.7702
  Mean fit time: 3.56s

Decision Tree (TfidfVectorizer):
  Best params: {'classifier__max_depth': 200, 'classifier__min_samples_split': 90}
  Best cross-validation score: 0.7601
  Mean fit time: 4.03s

Naive Bayes (CountVectorizer):
  Best params: {'classifier__alpha': 0.005}
  Best cross-validation score: 0.8275
  Mean fit time: 0.44s

Naive Bayes (TfidfVectorizer):
  Best params: {'classifier__alpha': 0.004}
  Best cross-validation score: 0.8258
  Mean fit time: 0.44s



,best_params,best_score,mean_fit_time
model,,,
Logistic Regression (CountVectorizer),{'classifier__C': 0.3},0.841023,0.544951
Logistic Regression (TfidfVectorizer),{'classifier__C': 0.15},0.843861,0.533759
Decision Tree (CountVectorizer),"{'classifier__max_depth': 250, 'classifier__min_samples_split': 70}",0.770210,3.557001
Decision Tree (TfidfVectorizer),"{'classifier__max_depth': 200, 'classifier__min_samples_split': 90}",0.760110,4.030077
Naive Bayes (CountVectorizer),{'classifier__alpha': 0.005},0.827548,0.442591
Naive Bayes (TfidfVectorizer),{'classifier__alpha': 0.004},0.825785,0.443209
